In [9]:
import numpy as np
import magpylib as magpy
from magpylib.magnet import Cuboid, Cylinder, CylinderSegment
from maggeometry import halbach_cylinder
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize, Bounds
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_grid

### Nelder-Mead Adaptive Optimization of Halbach 11 main
n = 11

### Samwise

In [10]:
# def halbach_cylinder(Br, c_d, c_h, D, n, alternate=False):

In [11]:
def get_grid_mag_and_target_nonuniformity(magnets, grid, grid_res, target=10, use_z=False):
    mT_to_G = 10
    B = magnets.getB(grid)
    G = B * mT_to_G
    # find magnitude of the b-field at all points over grid
    Gmag = 0
    if use_z:
        Gmag = G[:,:,2]
    else:
        Gmag = np.linalg.norm(G, axis=2)
#     Gmag = np.sqrt(G[:,:,0]**2 + G[:,:,1]**2 + G[:,:,2]**2)
#     Gmag = G[:,:,2]
    # find magnitude of b-field at center of grid
    mid_id = int(grid_res/2)
    Gcenter = Gmag[mid_id][mid_id]
    # calculate nonuniformity with respect to center b-field
    Gnon = np.abs((Gmag - Gcenter)/Gcenter)
    Gtarget_non = np.abs((Gmag - target)/target)
#     Gtarget_non = np.abs((Gmag - target)/target)
#     max_nonuniformity = np.max(Gnon)
    av_nonuniformity = np.sum(Gnon) / grid_res**2
    av_target_nonuniformity = np.sum(Gtarget_non) / grid_res**2
#     return Gmag, Gnon, Gcenter, av_nonuniformity, av_target_nonuniformity, max_nonuniformity
    return Gcenter, av_nonuniformity, av_target_nonuniformity

In [12]:
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_xy_grid

In [13]:
n = 11
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
Br = 1.09e3
c_h = 640
threshold = 0.1 # in G
target = 10

          #c_d,     D
bounds = ((1, 20), (21, 300))

def obj1_nonuniformity(x):
    magnets = halbach_cylinder(Br, x[0], c_h, x[1], n)
    center_field, av_nonuniformity, av_target_nonuniformity = get_grid_mag_and_target_nonuniformity(magnets, grid, grid_res, target)
    if (abs(center_field - target) > threshold):
        return av_target_nonuniformity
    return av_nonuniformity

In [ ]:
options = {'disp': True, 'fatol': 1e-18, 'maxiter': 5e3, 'adaptive': True}
method = "Nelder-Mead"

shots = 25
results_x = []
results_fun = []
guesses_x0 = []
b_len = len(bounds)
for i in range(shots):
    x0 = [0, 0]
    for b in range(b_len):
        x0[b] = np.random.uniform(low=bounds[b][0], high=bounds[b][1])
    print(x0)
    res = minimize(obj1_nonuniformity, x0, method=method, options=options, bounds=bounds)
    guesses_x0.append(x0)
    results_x.append(res.x)
    results_fun.append(res.fun)

print("Objective function values:")
print(results_fun)
print("Result parameters:")
print(results_x)
print("Starting guesses:")
print(guesses_x0)

[6.5202478832587465, 249.99166540121956]
Optimization terminated successfully.
         Current function value: 0.000027
         Iterations: 99
         Function evaluations: 244
[8.681745209173073, 260.31256544285503]


/var/folders/06/7pjfbw510cz2ch1m7lx8vvl80000gn/T/ipykernel_22091/1123013417.py:14: RuntimeWarning: Maximum number of iterations has been exceeded.
  res = minimize(obj1_nonuniformity, x0, method=method, options=options, bounds=bounds)


[3.7799962890062755, 272.6927225361992]


In [8]:
print(results_x)

[array([ 1.        , 69.68782727]), array([ 1.0000034 , 69.68674011]), array([ 1.       , 69.6883995]), array([  4.33272784, 300.        ]), array([  4.33206247, 300.        ]), array([ 1.        , 69.68773295]), array([ 1.        , 69.68747801]), array([  4.33230198, 300.        ]), array([ 1.        , 69.68811844]), array([ 1.        , 69.68864234]), array([ 1.        , 69.68745386]), array([ 1.        , 69.68667842]), array([ 1.        , 69.68842267]), array([ 1.        , 69.68668206]), array([  3.72004818, 258.24006578]), array([ 1.        , 69.68674039]), array([  4.33288864, 300.        ]), array([  4.33275576, 300.        ]), array([  4.33280204, 300.        ]), array([ 1.00000121, 69.6874346 ]), array([  4.3327065, 300.       ]), array([  4.33265914, 300.        ]), array([ 1.        , 69.68670307]), array([ 1.        , 69.68763613]), array([  4.33244608, 300.        ])]


In [9]:
charr = np.full((1, shots), 640).flatten()
r_n = np.full((1, shots), n).flatten()

In [10]:
from maggeometry import halbach_cylinder
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_xy_grid, get_grid_mag_and_nonuniformity
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
results_g_center = []
results_nonun = []
for x in results_x:
    magnets = halbach_cylinder(Br, x[0], charr[0], x[1], n)
    Gmag, Gnon, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_mag_and_nonuniformity(magnets, grid, grid_res)
    results_g_center.append(center_field)
    results_nonun.append(av_nonuniformity)

In [11]:
import pandas as pd
def make_opt_res_halbach_csv(name, fun, g_center, r_n, r_ch, x, guesses):
    r_f = np.array([fun])
    r_g = np.array(guesses)
    r_gc = np.array([g_center])
    r_n = np.array([r_n])
    r_ch = np.array([r_ch])
    r_x = np.array(x)
    a = np.concatenate((r_f.T, r_gc.T, r_n.T, r_ch.T, r_x, r_g), axis=1)
    b = a[a[:, 0].argsort()]

    columns = ['nonuniformity', 'center_field_gauss', 'n_rods', 'cyl_height']

    columns.append('cyl_diameter')
    columns.append('D')
    columns.append('g_cyl_diameter')
    columns.append('g_D')
    
    
    columns = np.array(columns)
    b_df = pd.DataFrame(b, columns = columns)
    
    c = np.concatenate((r_f.T, r_x), axis=1)
    new_array = [tuple(row) for row in c]
    u = np.unique(new_array, axis=0)
    n_unique = len(u)
    print('Number of unique results:', n_unique)
    print(b_df)
    b_df.to_csv(name + str(n_unique) + '.csv')
    return b, b_df

import datetime
today = datetime.datetime.today().strftime('%Y_%m_%d')

b, b_df = make_opt_res_halbach_csv(f'results/halbach/samwise/{n}_{today}_samwise_halbach_main_1090mT_', results_fun, results_g_center, r_n, charr, results_x, guesses_x0)

Number of unique results: 25
    nonuniformity  center_field_gauss  n_rods  cyl_height  cyl_diameter  \
0    1.469693e-07           10.099898     9.0       640.0      1.000000   
1    1.469737e-07           10.099892     9.0       640.0      1.000000   
2    1.469738e-07           10.099674     9.0       640.0      1.000000   
3    1.469771e-07           10.099482     9.0       640.0      1.000000   
4    1.469797e-07           10.099881     9.0       640.0      1.000000   
5    1.469834e-07           10.099950     9.0       640.0      1.000003   
6    1.469849e-07           10.099400     9.0       640.0      1.000000   
7    1.469849e-07           10.099899     9.0       640.0      1.000000   
8    1.469882e-07           10.099330     9.0       640.0      1.000000   
9    1.469885e-07           10.099593     9.0       640.0      1.000000   
10   1.469915e-07           10.099704     9.0       640.0      1.000001   
11   1.469917e-07           10.099621     9.0       640.0      1.000000

In [12]:
m = halbach_cylinder(Br, 1.101233, 640, 68.228286, n)
Gmag, Gnon, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_mag_and_nonuniformity(m, grid, grid_res)
print(av_nonuniformity)

1.5573750827657786e-07
